In [1]:
import yt
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm
from matplotlib import colormaps
import scipy
import pandas as pd
import os
import glob
from astropy.cosmology import FlatLambdaCDM
import pickle
import trident

In [2]:
def read_info(RamsesDir, timestep, alt_path=None):
    """ Returns info from the simulation (dict). """
    #savepath = get_alt_path(RamsesDir, alt_path=alt_path)
    #check_ratapath(savepath)
    savepath = RamsesDir
    path_info = f'{savepath}/ratadat/info_dict.pkl'
    if not os.path.exists(path_info):                # If first time, save general data
        info_dict = read_info_glob(RamsesDir, timestep=timestep)
        with open(path_info, 'wb') as file: 
            pickle.dump({'00000': info_dict}, file)
    with open(path_info, 'rb') as file:              # Otherwise, load common data
        all_info = pickle.load(file)
        info_dict = all_info.get('00000', {}).copy() # Important to copy, otherwise '00000' will be modified
        if timestep != 0:
            t_group = f'{timestep:05d}'
            if t_group in all_info:                  # And add timestep specific data.
                info_dict = all_info[t_group]
            else:
                info_dict = read_info_tstep(RamsesDir, timestep, info_dict)
                all_info[t_group] = info_dict
                with open(path_info, 'wb') as file:
                    pickle.dump(all_info, file)
    return info_dict


def read_info_glob(RamsesDir, timestep=1):
    """ Read info parameters common to all timesteps of a given simulation. """
    tstep = f'{timestep:05d}'
    path_out = f'{RamsesDir}/output_{tstep}/'
    
    # Initialise some variables which might not be defined.
    info_dict = {}
    info_dict['new_format'] = get_format(f'{path_out}header_{tstep}.txt')                     # Get whether post-2017 or not
    for key in ['isH2', 'delayed_cooling', 'momentum_feedback']:
        info_dict[key] = False

    # Namelist
    info_dict  = param2dict(info_dict, get_nml(RamsesDir, timestep))                                    # Namelist
    info_dict  = param2dict(info_dict, f'{path_out}info_rt_{tstep}.txt')                                # info_rt_xxxxx.txt
    info_dict  = hydro2dict(info_dict, f'{path_out}hydro_file_descriptor.txt', info_dict['new_format']) # hydro_file_descriptor
    info_dict  = get_nvar  (info_dict, path_out, tstep)                                                 # (after hydro2dict)

    # Corrections
    info_dict['X_fraction'] = round(info_dict['X_fraction'], 6) # Correct precision error in info_rt_xxxxx.txt
    info_dict['Y_fraction'] = round(info_dict['Y_fraction'], 6)

    # Determine simulation type
    # Note: original way is the following BUT have to read info_xxxxx.txt (resp header_xxx.txt), and some variables there depends on timestep
    #       info_dict['is_cosmo'] = info_dict['omega_m']!=1.0 and info_dict['aexp']!=1 and info_dict['H0']!=1 
    #       info_dict['is_zoom']  = info_dict['is_cosmo'] and info_dict['ndm'] != (2*info_dict['levelmin'])*info_dict['ndim']
    info_dict['is_cosmo'] = info_dict['cosmo']
    info_dict['is_zoom']  = info_dict['is_cosmo'] and 'initfile(2)' in info_dict
    info_dict['is_cool_refine'] = 'cooling_refine' in info_dict and any(value != -1 for value in info_dict['cooling_refine'])
    info_dict['nvarnoadvect'] = info_dict['cooling_time_ivar'] if info_dict['is_cool_refine'] else 0

    return info_dict


def read_info_tstep(RamsesDir, timestep, info_dict):
    """ Read info parameters for a given timestep anc add some units. """
    tstep = f'{timestep:05d}'
    path_out = f'{RamsesDir}/output_{tstep}/'
    info_dict  = param2dict(info_dict, f'{path_out}info_{tstep}.txt')               # info_xxxxx.txt
    info_dict  = headr2dict(info_dict, f'{path_out}header_{tstep}.txt', info_dict['new_format']) # header_xxxxx.txt

    # Units
    info_dict['unit_m']      = info_dict['unit_d']*info_dict['unit_l']**3
    info_dict['unit_v']      = info_dict['unit_l']/info_dict['unit_t']
    info_dict['unit_P']      = info_dict['unit_d']*info_dict['unit_v']**2
    info_dict['unit_T2']     = mH/kB*info_dict['unit_v']**2
    info_dict['unit_nH']     = info_dict['X_fraction']*info_dict['unit_d']/mH
    info_dict['unit_nHe']    = info_dict['unit_nH'] * info_dict['Y_fraction']/info_dict['X_fraction'] * 0.25
    info_dict['cu2cm']       = info_dict['unit_l'] * info_dict['boxlen']
    info_dict['boxlen_cMpc'] = info_dict['boxlen']*info_dict['unit_l']/info_dict['aexp']/kpc2cm/1e3 if info_dict['is_cosmo'] else None
    info_dict['redshift']    = 1./info_dict['aexp']-1
    info_dict['t_myr']       = info_dict['time']*info_dict['unit_t']/Myr2s if not info_dict['is_cosmo']\
        else FlatLambdaCDM(H0=info_dict['H0'],Om0=info_dict['omega_m']).age(1/info_dict['aexp']-1).value*1e3
    return info_dict


def get_nml(RamsesDir, timestep):
    """Return the path to the nml."""
    # Check the output directory
    path_nml = f'{RamsesDir}/output_{timestep:05d}/namelist.txt'    
    if os.access(path_nml, os.R_OK):            # Check it's readable
        with open(path_nml, 'r') as file:
            for line in file:
                if '&RUN_PARAMS' in line:       
                    return path_nml
    # If the file is doesn't exist/is corrupted, check the RamsesDir
    nml_files = glob.glob(f"{RamsesDir}/*.nml")
    path_nml = next((f for f in nml_files if os.access(f, os.R_OK)), None) # First nml with read access
    if path_nml is None:
        raise Exception(f"No accessible .nml file found in {RamsesDir}.")
    return path_nml


def headr2dict(info_dict, file_path, new_format):
    """ Add header variables (i.e. number of particles and particle fields) to info_dict. """
    header = {
        "Total number of particles": "npart",       # Old format
        "Total number of dark matter particles": "ndm",
        "Total number of star particles": "nstar",
        "Total number of sink particles": "nsink",
        "DM": "ndm",                                # New format
        "star": "nstar",
    }
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if "Particle fields" in line:
                info_dict["particle_fields"] = next(file).strip().split()
                break  # No more info after particle fields
            parts = line.split()
            if new_format and len(parts) == 2 and parts[0] in header:
                info_dict[header[parts[0]]] = int(parts[1])
            elif not new_format and 'Total' in line:
                info_dict[header[line]] = int(next(file).strip())
    return info_dict


def hydro2dict(info_dict, path_hydrofd, new_format):
    """ Read hydro_file_descriptor and add the indexes of the hydro parameters to info_dict. """
    with open(path_hydrofd, 'r') as file:
        for line in file:
            line = line.strip()
            if not line.startswith('#'): # skip the first in new format
                parts = line.split(',') if new_format else line.replace('#', ':').split(':')
                index1, index2 = (0, 1) if new_format else (-2, -1)
                if len(parts) >= 2:
                    info_dict[parts[index2].strip()] = int(parts[index1].strip())
    rename_map = {
        'density': 'idens',
        'velocity_x': 'ivel',
        'thermal_pressure': 'iPT',
        'pressure': 'iPT'
    }
    for old_name, new_name in rename_map.items():
        if old_name in info_dict:
            info_dict[new_name] = info_dict.pop(old_name)
    return info_dict


def get_nvar(info_dict, path_out, tstep):
    """ Try to get nvar from hydro_xxxxx.out00001, and in the hydro_file_descriptor if it fails. 
        First option is favoured as the hydro_file_descriptor can be badly coded. """
    hydro_file = f'{path_out}hydro_{tstep}.out00001'
    if os.path.exists(hydro_file):          # Try to read hydro_xxxxx.out00001
        with open(f'{path_out}hydro_{tstep}.out00001', 'rb') as f:  
            info_dict['nvar'] = np.fromfile(f, dtype=np.int32, count=5)[4]
    elif not 'dm' in path_out.lower():      # Else, skip if DM-only simulation and take the highest value from the hydro_file_descriptor.
        info_dict['nvar'] = info_dict[max((key for key in info_dict if 'scalar_' in key), key=lambda x: int(x.rsplit('_', 1)[1]), default=None)]
    return info_dict
def get_ramses_index(info, include_rt=False):                                                         # TODO: CR, MHD
    """ 
    Determine the index of the variables in the ramses output.
    """
    # Variables obtained in info
    dict_index = {'Density':       info['idens'],
                  'x-velocity':       info['ivel'],
                  'y-velocity':       info['ivel']+1                    if info['ndim']>=2 else None,
                  'z-velocity':       info['ivel']+2                    if info['ndim']>=3 else None,
                  'Pressure':        info['iPT']+info['rt_isIRtrap'],
                  'Metallicity':        info['iPT']+info['rt_isIRtrap']+1 if info['metal']   else None,
                  'xHI':      info['iIons']                     if info['isH2']    else None,
                  'xHII':     info['iIons']+info['isH2']        if info['rt']      else None,
                  'xHeII':    info['iIons']+info['isH2']+1      if info['rt']      else None,
                  'xHeIII':   info['iIons']+info['isH2']+2      if info['rt']      else None,
                }

    # Additional variables
    counter = info['iPT'] + info['rt_isIRtrap'] + info['metal'] + info['nIons']  # info['isH2'] already counted in nIons
    if info['delayed_cooling']:   dict_index['DC_var'],         counter = counter + 1, counter + 1  # delayed cooling
    if info['momentum_feedback']: dict_index['KR_turb'],        counter = counter + 1, counter + 1  # patch mom2 by Kretschmer
    if info['is_zoom']:           dict_index['zoom_var'],       counter = counter + 1, counter + 1  # zoom-in simulations
    if info['is_cool_refine']:    dict_index['cooling_length'], counter = counter + 1, counter + 1  # cooling length refinement

    # RT variables: N_photons and flux_nx/y/z for each group n.
    if info['rt'] and include_rt:
        for i in range(1, info['nGroups'] + 1):
            dict_index[f'N_photons{i}'] = info['nvar'] + 1 + (i - 1) * 4 if info['nGroups'] >= i else None
            for axis in ['x', 'y', 'z']:
                dict_index[f'flux_{i}{axis}'] = info['nvar'] + 2 + "xyz".index(axis) + (i - 1) * 4 if info['nGroups'] >= i else None

    return dict_index

def get_values(var, dictionary):    # TODO: restrict to one value, not T or temperature. Then, remove this function -> everything will be more readable.
    """ Helper function to be able to format dictionary keys as tuples. """
    for key in dictionary:
        if var in key:
            return dictionary[key]
    return None


def get_format(path_hdr):
    """ Returns whether the code is formatted following RAMSES post-2017 or not. """
    with open(path_hdr, 'r') as file:
        first_line = file.readline().strip()
        if 'Total number of particles' in first_line: return False
        elif '#      Family     Count' in first_line: return True
        else: raise ValueError("Unrecognized header format.")


def fort2py(value):
    """ Convert Fortran values to Python. """
    if value.lower() in ('.true.', '.false.'):  return value.lower() == '.true.'    # Booleans
    elif (value.startswith("'") and value.endswith("'")): return value[1:-1]        # Strings
    else:
        try: return float(value) if 'e' in value or '.' in value else int(value)    # Numerals
        except ValueError:  return value                                            # Other strings


def param2dict(info_dict, path_file):
    """ General function to get parameters from a file and add them to a dictionary. """
    with open(path_file, 'r') as file:
        for line in file:
            line = line.split('!')[0].strip()       # Remove comments and whitespace
            if '=' in line:
                key, value = map(str.strip, line.split('='))
                if key!='movie_vars_txt':           # Replace 'd' with 'e' for float compatibility
                    value = value.replace('d', 'e')
                if ',' in value or '*' in value:    # Convert lists
                    value_list = []
                    for element in value.split(','):
                        if '*' in element:
                            count, val = element.split('*')
                            value_list.extend([int(float(val.strip()))] * int(count))
                        else:
                            value_list.append(fort2py(element.strip()))
                    value = value_list
                else:
                    value = fort2py(value)          # Convert single values
                info_dict[key] = value
            elif "DOMAIN" in line or "Photon group properties" in line:
                break
    return info_dict

In [9]:
#This is just loading the simulation file
RamsesDir = '/minkewhale/kimm/rey/G8_normal/GTT_9pc'
timestep = 203

info = read_info(RamsesDir, 203)
dict_index = get_ramses_index(info)
print(dict_index)

extra_particle_fields = [("particle_birth_time", "float64"), ("particle_metallicity", "float64"), ("particle_imass", "float64")]
    #extra fields included in the data that yt does not automatically detect (I assume to save time?)
center_dat=[0.5, 0.5, 0.5]
rad_dat = 0.2739726724728318
bbox = [[c-rad_dat for c in center_dat], [c+rad_dat for c in center_dat]]
ds = yt.load(f'{RamsesDir}/output_{timestep:05d}', extra_particle_fields=extra_particle_fields, fields = list(dict_index.keys()), bbox = bbox)
ad = ds.all_data() #apparently required to put all data into a readable way.
#reg = ds.r[(-41.0959, "kpc"):(41.0959, "kpc"), (-41.0959, "kpc"):(41.0959, "kpc"), (-41.0959, "kpc"):(41.0959, "kpc")]
#specify where the ray would go to and from


yt : [INFO     ] 2024-07-26 15:01:24,802 Parameters: current_time              = 33.8546907656104
yt : [INFO     ] 2024-07-26 15:01:24,803 Parameters: domain_dimensions         = [128 128 128]
yt : [INFO     ] 2024-07-26 15:01:24,804 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2024-07-26 15:01:24,804 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2024-07-26 15:01:24,805 Parameters: cosmological_simulation   = 0


{'Density': 1, 'x-velocity': 2, 'y-velocity': 3, 'z-velocity': 4, 'Pressure': 5, 'Metallicity': 6, 'xHI': 7, 'xHII': 8, 'xHeII': 9, 'xHeIII': 10}


yt : [WARNING  ] 2024-07-26 15:01:26,614 This cooling file format is no longer supported. Cooling field loading skipped.


In [118]:
#This should add the ions estimates based on trident's integrated CLOUDY look up table
#Needs to be included in the line_list to be added to lines!!!!!!!
trident.add_ion_fields(ds, ions=['O VI'])

trident.add_ion_fields(ds, ions=['C IV'])

trident.add_ion_fields(ds, ions=['Mg II'])

yt : [WARNING  ] 2024-07-22 11:28:32,912 This cooling file format is no longer supported. Cooling field loading skipped.


In [119]:
#Conversion of units
def to_cl(kpc):
    return kpc*3.08567758128E+21/(0.150000000000000E+03*0.308567758128200E+22) #have to change accordingly with the specific file

In [120]:
#specifying impact parameter (kpc)
b = 10

In [121]:
#specifying what lines we want in our spectrum & create ray object
line_list = ['H', 'C', 'N', 'O', 'Mg', 'Si']

#creating a LightRay object that saves to a file called ray.h5 (HDF 5 file)
#NOTE: I modified the left_edge to copy on the light_ray.py from trident (line 471)

ray_start = np.array([0.5-0.2739726724728318, 0.5+to_cl(b), 0.5])
ray_end = np.array([0.5+0.2739726724728318, 0.5+to_cl(b), 0.5])
ray = trident.make_simple_ray(ds,
                              start_position=ray_start,
                              end_position=ray_end,
                              data_filename="ray.h5",
                              lines=line_list)
#ad = ray.all_data()



yt : [INFO     ] 2024-07-22 11:28:33,692 Getting segment at z = 0.0: [0.22602733 0.56666667 0.5       ] unitary to [0.77397267 0.56666667 0.5       ] unitary.
yt : [INFO     ] 2024-07-22 11:28:33,699 Getting subsegment: [0.22602733 0.56666667 0.5       ] unitary to [0.77397267 0.56666667 0.5       ] unitary.


  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



yt : [INFO     ] 2024-07-22 11:28:34,339 Saving field data to yt dataset: ray.h5.
yt : [INFO     ] 2024-07-22 11:28:34,378 Parameters: current_time              = 33.8546907656104 code_time
yt : [INFO     ] 2024-07-22 11:28:34,382 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-07-22 11:28:34,386 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2024-07-22 11:28:34,389 Parameters: domain_right_edge         = [1. 1. 1.] code_length
yt : [INFO     ] 2024-07-22 11:28:34,393 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2024-07-22 11:28:34,417 Allocating for 86 particles


In [122]:
#Finds the column density, for HI, it is pretty similar to Maxime Rey's thesis
column_density_HI = ad[('gas', 'Mg_p1_number_density')] * ad[('gas', 'dl')]
print('Column Density = %g' % column_density_HI.sum())

Column Density = 0


In [123]:
sg = trident.SpectrumGenerator(lambda_min=1239.5, lambda_max=1240.5, dlambda=0.001)
sg.make_spectrum(ray, lines=['Mg II'])
sg.save_spectrum('spec_Mg2.txt')
sg.plot_spectrum('spec_Mg2.png')

sg = trident.SpectrumGenerator(lambda_min='auto', lambda_max='auto', dlambda=0.001)
sg.make_spectrum(ray, lines=['C IV'])
sg.save_spectrum('spec_C4.txt')
sg.plot_spectrum('spec_C4.png')

sg = trident.SpectrumGenerator(lambda_min=1030, lambda_max=1035, dlambda=0.001)
sg.make_spectrum(ray, lines=['O VI'])
sg.save_spectrum('spec_O6.txt')
sg.plot_spectrum('spec_O6.png')

yt : [INFO     ] 2024-07-22 11:28:35,441 Setting instrument to Custom
yt : [INFO     ] 2024-07-22 11:28:35,455 Creating spectrum
yt : [INFO     ] 2024-07-22 11:28:35,467 Not adding line Mg II 1240: insufficient column density
yt : [INFO     ] 2024-07-22 11:28:35,470 Writing spectrum to ascii file: spec_Mg2.txt.
yt : [INFO     ] 2024-07-22 11:28:35,501 Writing spectrum plot to png file: spec_Mg2.png
yt : [INFO     ] 2024-07-22 11:28:35,599 Setting instrument to Custom
yt : [INFO     ] 2024-07-22 11:28:35,608 Creating spectrum
yt : [INFO     ] 2024-07-22 11:28:35,617 Not adding line C IV 1551: insufficient column density
yt : [INFO     ] 2024-07-22 11:28:35,618 Not adding line C IV 1548: insufficient column density
yt : [WARNING  ] 2024-07-22 11:28:35,620 Cannot add continuum with empty spectrum and lambda_min/max set to auto.
yt : [WARNING  ] 2024-07-22 11:28:35,621 Spectrum is totally empty!
yt : [WARNING  ] 2024-07-22 11:28:35,622 Spectrum is totally empty, no plotting to be done.
yt 

In [124]:
# making column densities graph as impact param (function)

def N_Mg (b):
    '''
    Function that finds the column density given an impact parameter, note for now light don't travel the same distance, this is to be altered
    
    Parameters:
        b (float): impact parameter in kpc above the y axis
    
    returns:
        float : the column density
    '''
    line_list = ['Mg']

    #creating a LightRay object

    ray_start = np.array([0, 0.5, 0.5+to_cl(b)])
    ray_end = np.array([1, 0.5, 0.5+to_cl(b)])
    ray = trident.make_simple_ray(ds,
                                start_position=ray_start,
                                end_position=ray_end,
                                data_filename="ray.h5",
                                lines=line_list)
    ad = ray.all_data()
    
    column_density_Mg = ad[('gas', 'Mg_p1_number_density')] * ad[('gas', 'dl')]
    return float(column_density_Mg.sum().value)

def N_C (b):
    '''
    Function that finds the column density given an impact parameter, note for now light don't travel the same distance, this is to be altered
    
    Parameters:
        b (float): impact parameter in kpc above the y axis
    
    returns:
        float : the column density
    '''
    line_list = ['C']

    #creating a LightRay object

    ray_start = np.array([0, 0.5, 0.5+to_cl(b)])
    ray_end = np.array([1, 0.5, 0.5+to_cl(b)])
    ray = trident.make_simple_ray(ds,
                                start_position=ray_start,
                                end_position=ray_end,
                                data_filename="ray.h5",
                                lines=line_list)
    ad = ray.all_data()
    
    column_density_C = ad[('gas', 'C_p3_number_density')] * ad[('gas', 'dl')]
    return float(column_density_C.sum().value)

def N_O (b):
    '''
    Function that finds the column density given an impact parameter, note for now light don't travel the same distance, this is to be altered
    
    Parameters:
        b (float): impact parameter in kpc above the y axis
    
    returns:
        float : the column density
    '''
    line_list = ['O']

    #creating a LightRay object

    ray_start = np.array([0, 0.5+to_cl(b), 0.5])
    ray_end = np.array([1, 0.5+to_cl(b), 0.5])
    ray = trident.make_simple_ray(ds,
                                start_position=ray_start,
                                end_position=ray_end,
                                data_filename="ray.h5",
                                lines=line_list)
    ad = ray.all_data()
    
    column_density_O = ad[('gas', 'O_p5_number_density')] * ad[('gas', 'dl')]
    return float(column_density_O.sum().value)

In [128]:
impact_param = np.arange(0, 60, 0.5)
C = []
for i in impact_param:
    C.append(N_C(i))

yt : [INFO     ] 2024-07-22 11:33:02,767 Getting segment at z = 0.0: [0.  0.5 0.5] unitary to [1.  0.5 0.5] unitary.
yt : [INFO     ] 2024-07-22 11:33:02,773 Getting subsegment: [0.  0.5 0.5] unitary to [1.  0.5 0.5] unitary.


  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



yt : [INFO     ] 2024-07-22 11:33:03,732 Saving field data to yt dataset: ray.h5.
yt : [INFO     ] 2024-07-22 11:33:03,827 Parameters: current_time              = 33.8546907656104 code_time
yt : [INFO     ] 2024-07-22 11:33:03,828 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-07-22 11:33:03,843 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2024-07-22 11:33:03,853 Parameters: domain_right_edge         = [1. 1. 1.] code_length
yt : [INFO     ] 2024-07-22 11:33:03,856 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2024-07-22 11:33:03,916 Allocating for 455 particles
yt : [INFO     ] 2024-07-22 11:33:04,054 Getting segment at z = 0.0: [0.         0.5        0.50333333] unitary to [1.         0.5        0.50333333] unitary.
yt : [INFO     ] 2024-07-22 11:33:04,056 Getting subsegment: [0.         0.5        0.50333333] unitary to [1.         0.5        0.50333333] unitary.


  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



yt : [INFO     ] 2024-07-22 11:33:04,755 Saving field data to yt dataset: ray.h5.
yt : [INFO     ] 2024-07-22 11:33:04,785 Parameters: current_time              = 33.8546907656104 code_time
yt : [INFO     ] 2024-07-22 11:33:04,786 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-07-22 11:33:04,786 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2024-07-22 11:33:04,787 Parameters: domain_right_edge         = [1. 1. 1.] code_length
yt : [INFO     ] 2024-07-22 11:33:04,788 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2024-07-22 11:33:04,801 Allocating for 288 particles
yt : [INFO     ] 2024-07-22 11:33:04,969 Getting segment at z = 0.0: [0.         0.5        0.50666667] unitary to [1.         0.5        0.50666667] unitary.
yt : [INFO     ] 2024-07-22 11:33:04,971 Getting subsegment: [0.         0.5        0.50666667] unitary to [1.         0.5        0.50666667] unitary.


...

yt : [INFO     ] 2024-07-22 11:34:16,343 Saving field data to yt dataset: ray.h5.
yt : [INFO     ] 2024-07-22 11:34:16,380 Parameters: current_time              = 33.8546907656104 code_time
yt : [INFO     ] 2024-07-22 11:34:16,382 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-07-22 11:34:16,385 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2024-07-22 11:34:16,387 Parameters: domain_right_edge         = [1. 1. 1.] code_length
yt : [INFO     ] 2024-07-22 11:34:16,390 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2024-07-22 11:34:16,408 Allocating for 128 particles
yt : [INFO     ] 2024-07-22 11:34:16,559 Getting segment at z = 0.0: [0.         0.5        0.89333333] unitary to [1.         0.5        0.89333333] unitary.
yt : [INFO     ] 2024-07-22 11:34:16,562 Getting subsegment: [0.         0.5        0.89333333] unitary to [1.         0.5        0.89333333] unitary.
yt : [INFO     ] 2024-07-22 11:34:16,758 Saving fi

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



yt : [INFO     ] 2024-07-22 11:34:16,790 Parameters: current_time              = 33.8546907656104 code_time
yt : [INFO     ] 2024-07-22 11:34:16,792 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-07-22 11:34:16,793 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2024-07-22 11:34:16,795 Parameters: domain_right_edge         = [1. 1. 1.] code_length
yt : [INFO     ] 2024-07-22 11:34:16,796 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2024-07-22 11:34:16,812 Allocating for 128 particles
yt : [INFO     ] 2024-07-22 11:34:16,993 Getting segment at z = 0.0: [0.         0.5        0.89666667] unitary to [1.         0.5        0.89666667] unitary.
yt : [INFO     ] 2024-07-22 11:34:16,997 Getting subsegment: [0.         0.5        0.89666667] unitary to [1.         0.5        0.89666667] unitary.


  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False



yt : [INFO     ] 2024-07-22 11:34:17,249 Saving field data to yt dataset: ray.h5.
yt : [INFO     ] 2024-07-22 11:34:17,288 Parameters: current_time              = 33.8546907656104 code_time
yt : [INFO     ] 2024-07-22 11:34:17,291 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2024-07-22 11:34:17,294 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2024-07-22 11:34:17,297 Parameters: domain_right_edge         = [1. 1. 1.] code_length
yt : [INFO     ] 2024-07-22 11:34:17,300 Parameters: cosmological_simulation   = 0
yt : [INFO     ] 2024-07-22 11:34:17,324 Allocating for 128 particles


In [130]:
plt.plot(impact_param, C, '.')
plt.xscale("log")
plt.yscale("log")

In [10]:
mu = ad[('gas', 'mu')]


[('gas', 'mu'), ('ramses-rt', 'xHI'), ('ramses-rt', 'xHII')]
YTRegion (info_00203): , center=[2.31425819e+23 2.31425819e+23 2.31425819e+23] cm, left_edge=[0. 0. 0.] cm, right_edge=[4.62851637e+23 4.62851637e+23 4.62851637e+23] cm
0 :  <yt.utilities.cython_fortran_utils.FortranFile object at 0x2ab129f501e0>
0 :  6
0 :  1
0 :  24
0 :  {'ncpu': 192, 'ndim': 3, 'nx': array([3, 3, 3], dtype=int32), 'nlevelmax': 14, 'ngridmax': 800000, 'nboundary': 6, 'ngrid_current': 26540, 'boxlen': 150.0, 'nout': array([202, 203, 204], dtype=int32), 'tout': array([ 0.16759124,  0.33518247,  0.50277371,  0.67036495,  0.83795619,
        1.00554743,  1.17313866,  1.3407299 ,  1.50832114,  1.67591237,
        1.84350361,  2.01109485,  2.17868609,  2.34627732,  2.51386856,
        2.6814598 ,  2.84905104,  3.01664227,  3.18423351,  3.35182475,
        3.51941599,  3.68700722,  3.85459846,  4.0221897 ,  4.18978094,
        4.35737218,  4.52496341,  4.69255465,  4.86014589,  5.02773712,
        5.19532836,  5.3

AttributeError: 'yt.utilities.cython_fortran_utils.FortranFile' object has no attribute 'name'